<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Closed_Water_Air.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [1]:
from dotmap import DotMap
import pandas as pd
import jax
from jax.lax import stop_gradient
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision
from tools.tree_array_transform import VSC, Comp, Range
import tools.che as che
R=8.314 # J/(mol K)

In [2]:
p = che.Props(['Nitrogen','Oxygen', 'Argon', 'Water'])

In [3]:
def model(c,r):
    # c: combination of adjustable variables and static state parameters
    # r: DotMap - store intermediate results for reporting
    r.Pw = p.Pvap(c.T)[3]
    r.P = c.air_n * R * c.T / c.V_vap + r.Pw
    return (c.P_f, r.P)

c=DotMap()
c.W_tot = 1. # 1 kg
c.V_tot = 0.01 # 10 Liters
c.P_i = 1e5 # Pa air pressure
c.P_f = 2e5 # Pa final pressure
c.T_i = 298.

c.Vw_i = c.W_tot/p.rhol(c.T_i)[3]
c.V_vap = c.V_tot - c.Vw_i # Approximation - water in the vapor phase is negligible
c.air_n = c.P_i * c.V_vap / (R * c.T_i)

c.T = Range(360,300,600)

/Users/carlosco/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [4]:
# transform our model into one that takes a single array of adjustable variables
# for minimization.  Unnecessary for this simple model with just 1 variable, 
# but very helpful for more complex problems later.
vsc=VSC(c,model)
vsc.solve()

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected.

in jax.numpy.split argument 1

While tracing the function hvp at /Users/carlosco/Documents/GitHub/CHE2064/tools/tree_array_transform.py:84, this value became a tracer due to JAX operations on these lines:

  operation cr:int32[1] = broadcast_in_dim[ broadcast_dimensions=(  )
                                shape=(1,) ] cq:int32[]
    from line /Users/carlosco/Documents/GitHub/CHE2064/tools/tree_array_transform.py:210 (unflatten)

  operation ct:int32[1] = concatenate[ dimension=0 ] cs:int32[0] cr:int32[1]
    from line /Users/carlosco/Documents/GitHub/CHE2064/tools/tree_array_transform.py:210 (unflatten)

  operation cv:int64[8] = gather[ dimension_numbers=GatherDimensionNumbers(offset_dims=(0,), collapsed_slice_dims=(), start_index_map=(0,))
                      slice_sizes=(8,) ] cu:int64[9] ct:int32[1]
    from line /Users/carlosco/Documents/GitHub/CHE2064/tools/tree_array_transform.py:210

In [5]:
# Dataframe of adjustable variables
vsc.vdf

,vector1
,1
T,365.719235


In [6]:
# Dataframe of intermediate results
vsc.rdf

,vector1
,1
Pw,77275.424480
P,200000.000101


In [41]:
# Dataframe of static state parameters
vsc.sdf

,vector1
,1
T_i,298.000000
P_f,200000.000000
V_vap,0.008995
V_tot,0.010000
W_tot,1.000000
air_n,0.363057
Vw_i,0.001005
P_i,100000.000000


In [42]:
# Raw values available as DotMaps
print(vsc.r) # intermediate results
print(vsc.v) # solution

#specific values
print(f'Temperature to achieve desired pressure: {vsc.v.T}')
print(f'Vapor pressure of water: {vsc.r.Pw}')
print(f'Volume of vapor phase: {vsc.r.V_vap}')

DotMap(Pw=DeviceArray(77275.42448034, dtype=float64), P=DeviceArray([200000.00010126], dtype=float64))
DotMap(T=DeviceArray([365.71923535], dtype=float64))
Temperature to achieve desired pressure: [365.71923535]
Vapor pressure of water: 77275.4244803356
Volume of vapor phase: DotMap()


In [47]:
# Set desired final pressure and moves of water in vapor phase
# Adjustable variables: final temperature and total container volume
def model2(c,r):
    r.Pw = p.Pvap(c.T)[3]
    r.V_vap = c.V_tot - c.Vw_i # Approximation - water in the vapor phase is negligible
    r.air_n = c.P_i * r.V_vap / (R * c.T_i)

    r.W_n_vap = r.Pw * r.V_vap / (R * c.T) 
    r.P = r.air_n  * R * c.T / r.V_vap + r.Pw

    # tuples from left and right side of equations
    P_constraint = (c.P_f, r.P)
    W_n_constraint = (r.W_n_vap, c.W_n_vap_desired)
    return (P_constraint, W_n_constraint)

In [48]:
c=DotMap()
c.W_tot = 1. # 1 kg
c.V_tot = 0.01 # 10 Liters
c.P_i = 1e5 # Pa air pressure
c.P_f = 2e5 # Pa final pressure
c.T_i = 298.
c.W_n_vap_desired = 0.3

c.Vw_i = c.W_tot/p.rhol(c.T_i)[3]
c.T = Range(350, 300, 400) #final temperature
c.V_tot = Range(0.015, 0., 0.03) # total container volume

vsc=VSC(c,model2)
vsc.solve(verbosity=0)

In [49]:
vsc.vdf

,vector1
,1
T,365.719235
V_tot,0.012809


In [50]:
vsc.rdf


,vector1
,1
Pw,77275.424383
V_vap,0.011804
air_n,0.476443
W_n_vap,0.300000
P,199999.999993


In [53]:
# The reduction in liquid volume of water due to evaporation is negligibly small
# usually.  Moles of water lost in vapor phase is set as an adjustable variable
# then constrained to be equal to that calculated from its vapor pressure and 
# volume of the vapor phase.

def model3(c, r):
    Pw = p.Pvap(c.T)[3]
    # volume of liquid water lost as water vapor ignored in model1
    # evaluate the density of water at T, but do not propagate gradients through.
    r.v_w_vap = c.W_n_vap*p.Mw[3]/1000/p.rhol(stop_gradient(c.T))[3]
    r.V_vap = c.V_tot - (c.Vw_i - r.v_w_vap)
    r.air_n = c.P_i * r.V_vap / (R * c.T_i)
    r.W_n_vap = Pw * r.V_vap / (R * c.T)
    P = (r.air_n + r.W_n_vap) * R * c.T / r.V_vap
    P_constraint = (c.P_f, P)
    W_constraint = (c.W_n_vap, r.W_n_vap)
    return (P_constraint,W_constraint)

In [54]:
c=DotMap()
c.W_tot = 1. # 1 kg
c.V_tot = 0.01 # 10 Liters
c.P_i = 1e5 # Pa air pressure
c.P_f = 2e5 # Pa final pressure
c.T_i = 298.
c.Vw_i = c.W_tot/p.rhol(c.T_i)[3]

c.T = Range(350., 300., 400.)
c.W_n_vap = Range(1., 0., 2.) # moles of water vapor

vsc=VSC(c,model3)
vsc.solve(verbosity=0)

In [55]:
vsc.rdf

,vector1
,1
v_w_vap,0.000004
V_vap,0.008999
air_n,0.363228
W_n_vap,0.228712


In [56]:
vsc.vdf

,vector1
,1
T,365.719235
W_n_vap,0.228712


In [60]:
# How does this affect the average molecular weight of the vapor phase?
print(f'MW of pure air: {jnp.sum(p.Mw[:3] * jnp.array([0.78, 0.21, 0.01]))}')

mass_air = jnp.sum(p.Mw[:3] * jnp.array([0.78, 0.21, 0.01]) * vsc.r.air_n)
mass_water_vapor = p.Mw[3] * vsc.r.W_n_vap
avg_MW = (mass_air+ mass_water_vapor)/(vsc.r.air_n + vsc.r.W_n_vap)
print(f'MW of air/water vapor phase: {avg_MW}')
# water vapor is lighter than air!

MW of pure air: 28.97019
MW of air/water vapor phase: [24.73735522]
